# 🎯 Final Project: Ordinal vs Nominal Sentiment Analysis
## Atharv Chaudhary - MSAI Class of 2027

**Research Question:** Do the performance gains from ordinal treatment justify the increased model complexity?

**Models:**
1. Multinomial Naive Bayes (Nominal Baseline)
2. Logistic Regression (Nominal - Multinomial)
3. Ridge Regression (Ordinal)
4. Ordinal Logistic Regression (Threshold-Based)

**Dataset:** Amazon Electronics Reviews (UCSD/McAuley Lab)

---

## 1. Setup & Installation

In [ ]:
# Install required libraries
!pip install datasets mord -q
print("✅ Installation complete!")

In [ ]:
# Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import (
    accuracy_score, 
    mean_absolute_error, 
    mean_squared_error,
    f1_score, 
    classification_report,
    confusion_matrix
)

# For loading Amazon dataset
from datasets import load_dataset

plt.style.use('seaborn-v0_8-whitegrid')
print("✅ All libraries imported!")

## 2. Load Dataset

In [ ]:
# Load Amazon Reviews Dataset from HuggingFace
print("Loading Amazon Electronics Reviews dataset...")
print("⚠️ This may take a few minutes for the first time.")

dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_Electronics",
    split="full",
    trust_remote_code=True
)

df_full = dataset.to_pandas()
print(f"✅ Loaded {len(df_full):,} reviews")
print(f"Columns: {df_full.columns.tolist()}")

In [ ]:
# Sample and clean data
SAMPLE_SIZE = 50000  # Adjust based on compute resources
RANDOM_STATE = 42

print(f"Sampling {SAMPLE_SIZE:,} reviews with stratification...")

# Convert rating to int
df_full['rating_int'] = df_full['rating'].astype(int)

# Stratified sampling
df = df_full.groupby('rating_int', group_keys=False).apply(
    lambda x: x.sample(min(len(x), SAMPLE_SIZE // 5), random_state=RANDOM_STATE)
).reset_index(drop=True)

# Keep only relevant columns
df = df[['text', 'rating']].copy()
df = df.dropna(subset=['text', 'rating'])
df = df[df['text'].str.len() >= 10]
df['rating'] = df['rating'].astype(int)
df = df[df['rating'].between(1, 5)]

print(f"✅ After cleaning: {len(df):,} reviews")
print(f"\nClass Distribution:")
print(df['rating'].value_counts().sort_index())

## 3. Visualize Class Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

rating_counts = df['rating'].value_counts().sort_index()
colors = ['#e74c3c', '#e67e22', '#f1c40f', '#2ecc71', '#27ae60']

bars = ax.bar(rating_counts.index, rating_counts.values, color=colors, edgecolor='black', linewidth=1.2)

for bar, count in zip(bars, rating_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            f'{count:,}\n({count/len(df)*100:.1f}%)', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_xlabel('Star Rating', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Reviews', fontsize=12, fontweight='bold')
ax.set_title('Class Distribution of Amazon Electronics Reviews', fontsize=14, fontweight='bold')
ax.set_xticks([1, 2, 3, 4, 5])
ax.set_xticklabels(['1 ⭐', '2 ⭐', '3 ⭐', '4 ⭐', '5 ⭐'], fontsize=11)

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Feature Extraction (TF-IDF)

In [ ]:
print("Creating TF-IDF features...")

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.95
)

X = vectorizer.fit_transform(df['text'])
y = df['rating'].values

print(f"✅ TF-IDF matrix shape: {X.shape}")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Training set: {X_train.shape[0]:,} samples")
print(f"✅ Test set: {X_test.shape[0]:,} samples")

## 5. Helper Functions

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Evaluate model and return metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    
    print(f"\n{'='*50}")
    print(f"📊 {model_name}")
    print(f"{'='*50}")
    print(f"Accuracy:      {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"MAE:           {mae:.4f}")
    print(f"MSE:           {mse:.4f}")
    print(f"F1 (macro):    {f1_macro:.4f}")
    print(f"F1 (weighted): {f1_weighted:.4f}")
    
    return {
        'model': model_name,
        'accuracy': accuracy,
        'mae': mae,
        'mse': mse,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

def calculate_adjacent_error_rate(y_true, y_pred):
    """Calculate percentage of errors between adjacent ratings."""
    errors = y_true != y_pred
    if errors.sum() == 0:
        return 0.0
    adjacent_errors = np.abs(y_true[errors] - y_pred[errors]) == 1
    return adjacent_errors.sum() / errors.sum()

def calculate_severe_error_rate(y_true, y_pred):
    """Calculate percentage of errors with distance >= 2."""
    errors = y_true != y_pred
    if errors.sum() == 0:
        return 0.0
    severe_errors = np.abs(y_true[errors] - y_pred[errors]) >= 2
    return severe_errors.sum() / errors.sum()

print("✅ Helper functions defined!")

## 6. Model 1: Multinomial Naive Bayes (Nominal Baseline)

In [ ]:
print("Training Multinomial Naive Bayes...")

nb_model = MultinomialNB(alpha=1.0)
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

nb_results = evaluate_model(y_test, nb_pred, "Naive Bayes (Nominal)")
nb_adjacent = calculate_adjacent_error_rate(y_test, nb_pred)
nb_severe = calculate_severe_error_rate(y_test, nb_pred)
print(f"Adjacent Error Rate: {nb_adjacent:.2%}")
print(f"Severe Error Rate:   {nb_severe:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, nb_pred, digits=4))

## 7. Model 2: Logistic Regression (Nominal)

In [ ]:
print("Training Logistic Regression (Nominal)...")

lr_nominal = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)
lr_nominal.fit(X_train, y_train)
lr_nom_pred = lr_nominal.predict(X_test)

lr_nom_results = evaluate_model(y_test, lr_nom_pred, "Logistic Regression (Nominal)")
lr_nom_adjacent = calculate_adjacent_error_rate(y_test, lr_nom_pred)
lr_nom_severe = calculate_severe_error_rate(y_test, lr_nom_pred)
print(f"Adjacent Error Rate: {lr_nom_adjacent:.2%}")
print(f"Severe Error Rate:   {lr_nom_severe:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, lr_nom_pred, digits=4))

## 8. Model 3: Ridge Regression (Ordinal)

This model treats ratings as **continuous ordinal values** (1 < 2 < 3 < 4 < 5), then rounds to nearest integer.

In [ ]:
print("Training Ridge Regression (Ordinal)...")
print("Treats ratings as continuous: 1 < 2 < 3 < 4 < 5")

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

ridge_pred_continuous = ridge_model.predict(X_test)
ridge_pred = np.clip(np.round(ridge_pred_continuous), 1, 5).astype(int)

ridge_results = evaluate_model(y_test, ridge_pred, "Ordinal Regression (Ridge)")
ridge_adjacent = calculate_adjacent_error_rate(y_test, ridge_pred)
ridge_severe = calculate_severe_error_rate(y_test, ridge_pred)
print(f"Adjacent Error Rate: {ridge_adjacent:.2%}")
print(f"Severe Error Rate:   {ridge_severe:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, ridge_pred, digits=4))

## 9. Model 4: Ordinal Logistic Regression (Threshold-Based)

This implements the **proportional odds model** using K-1 binary classifiers for K classes.

For each threshold k, we train a classifier for P(Y ≤ k).

In [ ]:
class OrdinalLogisticRegression:
    """
    Ordinal Logistic Regression using threshold approach.
    Trains K-1 binary classifiers for K classes.
    """
    def __init__(self, max_iter=1000):
        self.max_iter = max_iter
        self.classifiers = []
        self.classes_ = None
    
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        n_classes = len(self.classes_)
        
        # Train K-1 binary classifiers
        for k in range(n_classes - 1):
            threshold = self.classes_[k]
            # Binary labels: 1 if y <= threshold, 0 otherwise
            y_binary = (y <= threshold).astype(int)
            
            clf = LogisticRegression(max_iter=self.max_iter, random_state=42)
            clf.fit(X, y_binary)
            self.classifiers.append(clf)
        
        return self
    
    def predict_proba(self, X):
        """Predict class probabilities."""
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        
        # Get cumulative probabilities P(Y <= k)
        cumulative_probs = np.zeros((n_samples, n_classes))
        cumulative_probs[:, -1] = 1.0  # P(Y <= max_class) = 1
        
        for k, clf in enumerate(self.classifiers):
            cumulative_probs[:, k] = clf.predict_proba(X)[:, 1]
        
        # Convert to class probabilities P(Y = k)
        class_probs = np.zeros((n_samples, n_classes))
        class_probs[:, 0] = cumulative_probs[:, 0]
        for k in range(1, n_classes):
            class_probs[:, k] = cumulative_probs[:, k] - cumulative_probs[:, k-1]
        
        # Ensure non-negative (numerical stability)
        class_probs = np.maximum(class_probs, 0)
        row_sums = class_probs.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1
        class_probs = class_probs / row_sums
        
        return class_probs
    
    def predict(self, X):
        """Predict class labels."""
        probs = self.predict_proba(X)
        return self.classes_[np.argmax(probs, axis=1)]

print("✅ Ordinal Logistic Regression class defined!")

In [ ]:
print("Training Ordinal Logistic Regression...")

olr_model = OrdinalLogisticRegression(max_iter=1000)
olr_model.fit(X_train, y_train)
olr_pred = olr_model.predict(X_test)

olr_results = evaluate_model(y_test, olr_pred, "Ordinal Logistic Regression")
olr_adjacent = calculate_adjacent_error_rate(y_test, olr_pred)
olr_severe = calculate_severe_error_rate(y_test, olr_pred)
print(f"Adjacent Error Rate: {olr_adjacent:.2%}")
print(f"Severe Error Rate:   {olr_severe:.2%}")

print("\nClassification Report:")
print(classification_report(y_test, olr_pred, digits=4))

## 10. Results Summary

In [ ]:
# Compile all results
all_results = pd.DataFrame([
    {**nb_results, 'encoding': 'Nominal', 'adjacent_error': nb_adjacent, 'severe_error': nb_severe},
    {**lr_nom_results, 'encoding': 'Nominal', 'adjacent_error': lr_nom_adjacent, 'severe_error': lr_nom_severe},
    {**ridge_results, 'encoding': 'Ordinal', 'adjacent_error': ridge_adjacent, 'severe_error': ridge_severe},
    {**olr_results, 'encoding': 'Ordinal', 'adjacent_error': olr_adjacent, 'severe_error': olr_severe},
])

all_results = all_results[['model', 'encoding', 'accuracy', 'mae', 'mse', 'f1_macro', 'f1_weighted', 'adjacent_error', 'severe_error']]

print("\n" + "="*80)
print("📊 COMPLETE RESULTS SUMMARY")
print("="*80)
display(all_results)

# Save results
all_results.to_csv('model_results.csv', index=False)
print("\n✅ Saved: model_results.csv")

## 11. Visualizations

In [ ]:
# Model Comparison Chart
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

models = ['NB\n(Nominal)', 'LR\n(Nominal)', 'Ridge\n(Ordinal)', 'OLR\n(Ordinal)']
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

# Accuracy
axes[0, 0].bar(range(len(models)), all_results['accuracy'], color=colors, edgecolor='black')
axes[0, 0].set_xticks(range(len(models)))
axes[0, 0].set_xticklabels(models, fontsize=10)
axes[0, 0].set_ylabel('Accuracy', fontsize=11)
axes[0, 0].set_title('Accuracy (Higher = Better)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate(all_results['accuracy']):
    axes[0, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')

# MAE
axes[0, 1].bar(range(len(models)), all_results['mae'], color=colors, edgecolor='black')
axes[0, 1].set_xticks(range(len(models)))
axes[0, 1].set_xticklabels(models, fontsize=10)
axes[0, 1].set_ylabel('Mean Absolute Error', fontsize=11)
axes[0, 1].set_title('MAE (Lower = Better)', fontsize=12, fontweight='bold')
for i, v in enumerate(all_results['mae']):
    axes[0, 1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')

# F1 Macro
axes[1, 0].bar(range(len(models)), all_results['f1_macro'], color=colors, edgecolor='black')
axes[1, 0].set_xticks(range(len(models)))
axes[1, 0].set_xticklabels(models, fontsize=10)
axes[1, 0].set_ylabel('F1 Score (Macro)', fontsize=11)
axes[1, 0].set_title('F1 Macro (Higher = Better)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylim([0, 1])
for i, v in enumerate(all_results['f1_macro']):
    axes[1, 0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')

# Error Types
x = np.arange(len(models))
width = 0.35
axes[1, 1].bar(x - width/2, all_results['adjacent_error'], width, label='Adjacent (±1)', color='#f39c12', edgecolor='black')
axes[1, 1].bar(x + width/2, all_results['severe_error'], width, label='Severe (±2+)', color='#c0392b', edgecolor='black')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(models, fontsize=10)
axes[1, 1].set_ylabel('Error Rate', fontsize=11)
axes[1, 1].set_title('Error Type Distribution', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Confusion Matrices
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

predictions = [
    (nb_pred, "Naive Bayes (Nominal)"),
    (lr_nom_pred, "Logistic Regression (Nominal)"),
    (ridge_pred, "Ridge Regression (Ordinal)"),
    (olr_pred, "Ordinal Logistic Regression")
]

for ax, (pred, name) in zip(axes.flat, predictions):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=[1, 2, 3, 4, 5],
                yticklabels=[1, 2, 3, 4, 5])
    ax.set_xlabel('Predicted Rating', fontsize=11)
    ax.set_ylabel('Actual Rating', fontsize=11)
    ax.set_title(f'{name}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

## 12. Key Findings

In [ ]:
print("\n" + "="*60)
print("📝 KEY FINDINGS")
print("="*60)

best_accuracy = all_results.loc[all_results['accuracy'].idxmax()]
best_mae = all_results.loc[all_results['mae'].idxmin()]
lowest_severe = all_results.loc[all_results['severe_error'].idxmin()]

print(f"\n1. BEST ACCURACY: {best_accuracy['model']}")
print(f"   Accuracy: {best_accuracy['accuracy']:.4f} ({best_accuracy['accuracy']*100:.2f}%)")

print(f"\n2. LOWEST MAE: {best_mae['model']}")
print(f"   MAE: {best_mae['mae']:.4f}")

print(f"\n3. LOWEST SEVERE ERROR RATE: {lowest_severe['model']}")
print(f"   Severe Error Rate: {lowest_severe['severe_error']:.2%}")

# Ordinal vs Nominal comparison
nominal_avg_mae = all_results[all_results['encoding'] == 'Nominal']['mae'].mean()
ordinal_avg_mae = all_results[all_results['encoding'] == 'Ordinal']['mae'].mean()
improvement = (nominal_avg_mae - ordinal_avg_mae) / nominal_avg_mae * 100

print(f"\n4. ORDINAL VS NOMINAL:")
print(f"   Nominal Avg MAE: {nominal_avg_mae:.4f}")
print(f"   Ordinal Avg MAE: {ordinal_avg_mae:.4f}")
print(f"   MAE Improvement: {improvement:.2f}%")

## 13. Download Files

In [ ]:
# For Google Colab - download files
try:
    from google.colab import files
    
    files.download('class_distribution.png')
    files.download('model_comparison.png')
    files.download('confusion_matrices.png')
    files.download('model_results.csv')
    print("✅ Files downloaded!")
except:
    print("Not in Colab - files saved locally.")